In [ ]:
!pip install image-classifiers==1.0.0b1

In [ ]:
# Library imports
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from skimage.io import imread
from skimage.transform import resize
from sklearn.metrics import accuracy_score
import re
from classification_models.tfkeras import Classifiers

In [ ]:
# model/training variables. batch size must be divisor of number of training, validation, and test records.
BATCH_SIZE = 30

In [ ]:
# https://medium.com/@mrgarg.rajat/training-on-large-datasets-that-dont-fit-in-memory-in-keras-60a974785d71
class My_Custom_Generator(tf.keras.utils.Sequence) :
  
  def __init__(self, image_filenames, labels, batch_size, IMG_HEIGHT, IMG_WIDTH) :
    self.image_filenames = image_filenames
    self.labels = labels
    self.batch_size = batch_size
    self.IMG_HEIGHT = IMG_HEIGHT
    self.IMG_WIDTH = IMG_WIDTH
    
    
  def __len__(self) :
    return (np.ceil(len(self.image_filenames) / float(self.batch_size))).astype(np.int)
  
  
  def __getitem__(self, idx) :
    batch_x = self.image_filenames[idx * self.batch_size : (idx+1) * self.batch_size]
    batch_y = self.labels[idx * self.batch_size : (idx+1) * self.batch_size]
    
    return np.array([
            np.resize(imread(str(file_name)), (self.IMG_HEIGHT, self.IMG_WIDTH, 3))
               for file_name in batch_x])/255.0, np.array(batch_y)

In [ ]:
def load_data():
    with open('/content/drive/My Drive/Data/final-book30-labels-train.csv', mode='r', encoding='utf-8', errors='ignore') as f:
        train_labels = pd.read_csv(f, delimiter=",", header=None, names=['record', 'Filename', 'Category ID'])

    with open('/content/drive/My Drive/Data/final-book30-labels-valid.csv', mode='r', encoding='utf-8', errors='ignore') as f:
        valid_labels = pd.read_csv(f, delimiter=",", header=None, names=['record', 'Filename', 'Category ID'])

    train_labels = train_labels.assign(Full_Filename = '/content/padded/'+ train_labels["Filename"])
    valid_labels = valid_labels.assign(Full_Filename = '/content/valid_padded/padded/'+ valid_labels["Filename"])

    print('Loading data')
     # Load actual data
    zip_path = '/content/drive/My Drive/images/Train/padded.zip'
    !cp "{zip_path}" .
    !unzip -q "padded.zip" 
    !rm "padded.zip" 

    zip_path = '/content/drive/My Drive/images/Valid/padded.zip'
    !cp "{zip_path}" .
    !unzip -q "padded.zip"  -d "valid_padded" 
    !rm "padded.zip" 

    return train_labels, valid_labels


def train_model(base_model, IMG_HEIGHT, IMG_WIDTH, folder, train_labels, valid_labels):
    """
    Function to load all the relevant data, train the model with early stopping, saving best validation accuracy snapshots, and 
        save training results to a file for later visualisation. 
    arg:
        dataset - name of the dataset files to work from
        skip_load - should the load from drive into colab be skipped
    """
   
    print('Prepping model')
    my_training_batch_generator = My_Custom_Generator(train_labels["Full_Filename"], train_labels["Category ID"], BATCH_SIZE, IMG_HEIGHT, IMG_WIDTH)
    my_validation_batch_generator = My_Custom_Generator(valid_labels["Full_Filename"], valid_labels["Category ID"], BATCH_SIZE, IMG_HEIGHT, IMG_WIDTH)


    base_model.trainable = False

    model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(30, activation='softmax')
    ])

    model.compile(optimizer='Adam',
              loss='SparseCategoricalCrossentropy',
              metrics=['accuracy', tf.keras.metrics.SparseTopKCategoricalAccuracy(3)])
    
      #early stopping and checkpoints
    es = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', verbose=1, patience=20)
    mc = tf.keras.callbacks.ModelCheckpoint(f'/content/drive/My Drive/Models/{folder}/' + '{epoch:02d}.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True, save_weights_only=True)
    cl = tf.keras.callbacks.CSVLogger(f"/content/drive/My Drive/Models/{folder}/model_history_log.csv", append = True)

    epoch_files = os.listdir(f'/content/drive/My Drive/Models/{folder}/')
    pat = re.compile(f'^.*\.h5$')
    epoch_files_cut = [i for i in epoch_files if pat.match(i) ]
    if len(epoch_files_cut) > 0:
        model_weights = max(epoch_files_cut)
        epoch = int(os.path.basename(model_weights)[:-3])

        model.load_weights(f'/content/drive/My Drive/Models/{folder}/{model_weights}')
    else:
        epoch = 0

    print('Training model')
    history = model.fit_generator(generator=my_training_batch_generator, 
                              validation_data = my_validation_batch_generator,
                              steps_per_epoch = int(len(train_labels) // BATCH_SIZE),
                              validation_steps = int(len(valid_labels) // BATCH_SIZE),
                              initial_epoch = epoch,
                              epochs = 200,
                              verbose = 1,
                              callbacks = [es, mc, cl])
    
    return history


In [ ]:
!rm -r padded
!rm -r valid_padded

In [ ]:
train_labels, valid_labels = load_data()

Loading data


In [ ]:
## MobileNetV2
base_model = tf.keras.applications.MobileNetV2(input_shape=(224, 224, 3),
                                               include_top=False,
                                               weights='imagenet')
mob_hist = train_model(base_model, 224, 224, 'mobilenet', train_labels, valid_labels)

Prepping model
Training model
Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/1000
1539/1539 [==============================] - ETA: 0s - loss: 3.2132 - accuracy: 0.1213 - sparse_top_k_categorical_accuracy: 0.2613
Epoch 00001: val_accuracy improved from -inf to 0.14561, saving model to /content/drive/My Drive/Models/mobilenet/01.h5
1539/1539 [==============================] - 205s 133ms/step - loss: 3.2132 - accuracy: 0.1213 - sparse_top_k_categorical_accuracy: 0.2613 - val_loss: 3.1072 - val_accuracy: 0.1456 - val_sparse_top_k_categorical_accuracy: 0.2984
Epoch 2/1000
1539/1539 [==============================] - ETA: 0s - loss: 3.0491 - accuracy: 0.1574 - sparse_top_k_categorical_accuracy: 0.3205
Epoch 00002: val_accuracy improved from 0.14561 to 0.14795, saving model to /content/drive/My Drive/Models/mobilenet/02.h5
1539/1539 [==============================] - 201s 130ms/step - loss: 3.0491 - accuracy: 0.1574 - sparse_top_k_categorical_accuracy: 0.

KeyboardInterrupt: ignored

In [ ]:
## InceptionResnetV2
base_model = tf.keras.applications.InceptionResNetV2(input_shape=(299, 299, 3),
                                               include_top=False,
                                               weights='imagenet')
inc_hist = train_model(base_model, 299, 299, 'inception_resnetv2', train_labels, valid_labels)

Prepping model
Training model
Epoch 47/1000
1539/1539 [==============================] - ETA: 0s - loss: 2.2178 - accuracy: 0.3679 - sparse_top_k_categorical_accuracy: 0.5914
Epoch 00047: val_accuracy improved from -inf to 0.27212, saving model to /content/drive/My Drive/Models/inception_resnetv2/47.h5
1539/1539 [==============================] - 516s 336ms/step - loss: 2.2178 - accuracy: 0.3679 - sparse_top_k_categorical_accuracy: 0.5914 - val_loss: 2.7700 - val_accuracy: 0.2721 - val_sparse_top_k_categorical_accuracy: 0.4589
Epoch 48/1000
1539/1539 [==============================] - ETA: 0s - loss: 2.2197 - accuracy: 0.3658 - sparse_top_k_categorical_accuracy: 0.5924
Epoch 00048: val_accuracy did not improve from 0.27212
1539/1539 [==============================] - 511s 332ms/step - loss: 2.2197 - accuracy: 0.3658 - sparse_top_k_categorical_accuracy: 0.5924 - val_loss: 2.8252 - val_accuracy: 0.2618 - val_sparse_top_k_categorical_accuracy: 0.4468
Epoch 49/1000
1539/1539 [=============

In [ ]:
# Resnext, has to come from anotehr package
resnext50, preprocess_input = Classifiers.get('resnext50')
base_model = resnext50((224, 224, 3), weights='imagenet', include_top = False)

res_hist = train_model(base_model, 224, 224, 'resnext', train_labels, valid_labels)

Prepping model
Training model
Epoch 85/200
1539/1539 [==============================] - ETA: 0s - loss: 3.1520 - accuracy: 0.1258 - sparse_top_k_categorical_accuracy: 0.2725
Epoch 00085: val_accuracy improved from -inf to 0.12417, saving model to /content/drive/My Drive/Models/resnext/85.h5
1539/1539 [==============================] - 554s 360ms/step - loss: 3.1520 - accuracy: 0.1258 - sparse_top_k_categorical_accuracy: 0.2725 - val_loss: 3.2105 - val_accuracy: 0.1242 - val_sparse_top_k_categorical_accuracy: 0.2591
Epoch 86/200
1539/1539 [==============================] - ETA: 0s - loss: 3.1473 - accuracy: 0.1293 - sparse_top_k_categorical_accuracy: 0.2718
Epoch 00086: val_accuracy did not improve from 0.12417
1539/1539 [==============================] - 545s 354ms/step - loss: 3.1473 - accuracy: 0.1293 - sparse_top_k_categorical_accuracy: 0.2718 - val_loss: 3.2091 - val_accuracy: 0.1238 - val_sparse_top_k_categorical_accuracy: 0.2624
Epoch 87/200
1539/1539 [===========================

KeyboardInterrupt: ignored